In [8]:
import pandas as pd

# Especifica la ruta de tu archivo CSV
archivo_csv = 'captionings.csv'

# Carga el DataFrame desde el archivo CSV
df = pd.read_csv(archivo_csv)

# Muestra el DataFrame cargado
df


,img


In [ ]:
import os

# Especifica la ruta de tu carpeta de imágenes
ruta_carpeta = 'C:/Tesis/imagic-editing.github.io/tedbench/originals'

# Lista todos los archivos en la carpeta
archivos_en_carpeta = os.listdir(ruta_carpeta)

# Filtra solo los archivos con extensiones de imágenes comunes (puedes ajustar la lista según tus necesidades)
extensiones_imagen = ['.jpg', '.jpeg', '.png', '.gif']
imagenes = [archivo for archivo in archivos_en_carpeta if any(archivo.lower().endswith(ext) for ext in extensiones_imagen)]


df['img'] = imagenes

# Muestra el DataFrame resultante
df


# Captioning with BLIP2

In [10]:
#Captioning con BLIP2
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
)
model.to(device)

c:\Users\magr3\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [01:24<00:00, 42.34s/it]


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [11]:

def generar_caption(img_name):
    # Cargar la imagen desde la URL
    imagen = Image.open(ruta_carpeta + "/" + img_name)

    # Procesar la imagen y prepararla para el modelo
    inputs = processor(images=imagen, return_tensors="pt").to(device, torch.float16)

    # Generar el caption usando el modelo
    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    return generated_text

In [12]:
generar_caption("apples.jpeg")

c:\Users\magr3\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a basket filled with apples sitting on a table'

In [14]:
from tqdm import tqdm


In [15]:
tqdm.pandas(desc="Generando captions")
df['blip2'] = df['img'].progress_apply(generar_caption)
df

Generando captions:   0%|          | 0/40 [00:00<?, ?it/s]c:\Users\magr3\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generando captions: 100%|██████████| 40/40 [02:20<00:00,  3.51s/it]


,img,blip2
0,apples.jpeg,a basket filled with apples sitting on a table
1,banana_1.jpeg,a banana with black spots on it
2,bear3.jpeg,a black bear walking through a grassy field
3,bird-g83440b9c4_1920.jpg,two colorful birds sitting on a rope
4,bird.jpeg,"kingfisher, kingfisher, kingfisher, kingfisher..."
5,bird.png,"kingfisher, kingfisher, kingfisher, kingfisher..."
6,black_shirt.jpeg,a man in black t - shirt standing on a sidewalk
7,box.jpeg,a brown box with a knife on the floor
8,cake_1.jpeg,a chocolate cake with chocolate drips on top
9,cat.jpeg,a white cat sitting on the ground


In [16]:
df.to_csv(archivo_csv, index=False)